#使用Apriori算法进行关联分析

In [1]:
from numpy import *

def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return map(frozenset, C1)#use frozen set so we
                            #can use it as a key in a dict    

def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not ssCnt.has_key(can): ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [5]:
dataSet=loadDataSet()
print dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]


In [6]:
C1=createC1(dataSet)
print C1

[frozenset([1]), frozenset([2]), frozenset([3]), frozenset([4]), frozenset([5])]


In [7]:
D=map(set,dataSet)
print D

[set([1, 3, 4]), set([2, 3, 5]), set([1, 2, 3, 5]), set([2, 5])]


In [9]:
L1,suppData0=scanD(D,C1,0.5)
print L1

[frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])]


In [10]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = map(set, dataSet)
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [20]:
L,suppData=apriori(dataSet)
for each in L:
    print each

[frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])]
[frozenset([1, 3]), frozenset([2, 5]), frozenset([2, 3]), frozenset([3, 5])]
[frozenset([2, 3, 5])]
[]


In [21]:
aprioriGen(L[0],2)

[frozenset({1, 3}),
 frozenset({1, 2}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({3, 5}),
 frozenset({2, 5})]

In [22]:
L,suppData=apriori(dataSet,minSupport=0.6)
for each in L:
    print each

[frozenset([3]), frozenset([2]), frozenset([5])]
[frozenset([2, 5])]
[]
